In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferWindowMemory(
    llm=llm,
    return_message=True,
    k=2,
)

def add_message(request, answer):
    memory.save_context({"request": request}, {"answer": answer})

add_message(
    "I want the Titanic to be represented with 3 emojis",
    "Titanic -> 👩‍❤️‍👨🛳️☠️"
)
add_message(
    "I want the Harbin to be represented with 3 emojis",
    "Harbin -> 🚊🗾🔫"
)
add_message(
    "I want the 용감한 시민 to be represented with 3 emojis",
    "용감한 시민 -> 👩‍🏫👊🙇"
)

example_prompt = PromptTemplate.from_template("Human: {input}\nAI: {output}")

def memory_to_examples():
    examples = []
    for message in memory.chat_memory.messages:
        if message.type == "human":
            input = message.content
        elif message.type == "ai":
            output = message.content
            examples.append({"input": input, "output": output})
    return examples

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=memory_to_examples(),
    suffix="Human: I want the {moviename} to be represented with 3 emojis",
    input_variables=["moviename"]
)

chain = RunnablePassthrough.assign(history=memory.load_memory_variables) | prompt | llm

def answer_question(question):
    result = chain.invoke({"moviename": question})
    add_message(question, result.content)
    print(result.content)




In [8]:
answer_question("Frozen")

AI: Frozen -> ❄️👸⛄


In [9]:
answer_question("Your name")

AI: Your name -> 👫🌌📝


In [12]:
list(memory.load_memory_variables({}).items())[0]
#메모리 잘 저장되었나 확인
#window 크기를 2로 지정하고 2개의 새 입력을 만든 후
#첫번째 입력을 출력했을 때 기존에 입력되어 있던 예시는 사라지고
#첫번째 입력했던 값이 출력되었으므로 메모리로 관리된 데이터가 사용되고 있음을 알 수 있다..

('history',
 'Human: Frozen\nAI: AI: Frozen -> ❄️👸⛄\nHuman: Your name\nAI: AI: Your name -> 👫🌌📝')